In [3]:
import os
import sys
import json
import numpy as np
import skimage.draw
import cv2
from mrcnn.visualize import display_instances
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard

# Root directory of the project
ROOT_DIR = "C:\\Users\\Hon\\Documents\\skipsi\\ehon\\nyoba"

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils

# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

class CustomConfig(Config):
    """Configuration for training on the custom dataset."""
    NAME = "object"
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 3
    STEPS_PER_EPOCH = 70
    DETECTION_MIN_CONFIDENCE = 0.5
    VALIDATION_STEPS = 33
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512
    LEARNING_RATE = 0.001

class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset):
        """Load a subset of the custom dataset."""
        self.add_class("object", 1, "sawah")
        self.add_class("object", 2, "bangunan")
        self.add_class("object", 3, "landasan")

        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        annotations1 = json.load(open('C:\\Users\\Hon\\Documents\\skipsi\\ehon\\Skripsi-Hon\\Dataset\\train\\train.json'))
        annotations = list(annotations1.values())
        annotations = [a for a in annotations if a['regions']]

        for a in annotations:
            polygons = [r['shape_attributes'] for r in a['regions']]
            objects = [s['region_attributes']['names'] for s in a['regions']]
            name_dict = {"sawah": 1, "bangunan": 2, "landasan": 3}
            num_ids = [name_dict[a] for a in objects]

            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path, plugin='matplotlib')
            height, width = image.shape[:2]

            self.add_image(
                "object",
                image_id=a['filename'],
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
            )

    def load_mask(self, image_id):
        """Generate instance masks for an image."""
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        info = self.image_info[image_id]
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])], dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1

        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

def train(model):
    dataset_train = CustomDataset()
    dataset_train.load_custom("Dataset", "train")
    dataset_train.prepare()

    dataset_val = CustomDataset()
    dataset_val.load_custom("Dataset", "val")
    dataset_val.prepare()
    

    log_dir = os.path.join(DEFAULT_LOGS_DIR, "tensorboard_logs")
    tensorboard_callback = TensorBoard(log_dir=log_dir)

    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=40,  # Increase epochs for better training
                layers='heads',
                custom_callbacks=[tensorboard_callback])

config = CustomConfig()
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=DEFAULT_LOGS_DIR)

weights_path = COCO_WEIGHTS_PATH
if not os.path.exists(weights_path):
    utils.download_trained_weights(weights_path)

model.load_weights(weights_path, by_name=True, exclude=[
    "mrcnn_class_logits", "mrcnn_bbox_fc",
    "mrcnn_bbox", "mrcnn_mask"])

train(model)


Training network heads

Starting at epoch 0. LR=0.001

Checkpoint Path: C:\Users\Hon\Documents\skipsi\ehon\nyoba\logs\object20240703T0035\mask_rcnn_object_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mas

C:\Users\Hon\anaconda3\envs\honn\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\Users\Hon\anaconda3\envs\honn\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\Users\Hon\anaconda3\envs\honn\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
70/70 [==============================] - 951s 14s/step - loss: 3.0543 - rpn_class_loss: 0.3900 - rpn_bbox_loss: 0.5930 - mrcnn_class_loss: 0.6554 - mrcnn_bbox_loss: 0.7246 - mrcnn_mask_loss: 0.6913 - val_loss: 2.1886 - val_rpn_class_loss: 0.2397 - val_rpn_bbox_loss: 0.4309 - val_mrcnn_class_loss: 0.4517 - val_mrcnn_bbox_loss: 0.5664 - val_mrcnn_mask_loss: 0.4999
Epoch 2/100
70/70 [==============================] - 924s 13s/step - loss: 2.0628 - rpn_class_loss: 0.2243 - rpn_bbox_loss: 0.4079 - mrcnn_class_loss: 0.4386 - mrcnn_bbox_loss: 0.5557 - mrcnn_mask_loss: 0.4362 - val_loss: 1.7582 - val_rpn_class_loss: 0.2034 - val_rpn_bbox_loss: 0.3769 - val_mrcnn_class_loss: 0.3247 - val_mrcnn_bbox_loss: 0.4745 - val_mrcnn_mask_loss: 0.3786
Epoch 3/100
70/70 [==============================] - 918s 13s/step - loss: 1.7897 - rpn_class_loss: 0.1909 - rpn_bbox_loss: 0.3666 - mrcnn_class_loss: 0.3872 - mrcnn_bbox_loss: 0.4740 - mrcnn_mask_loss: 0.3711 - val_loss: 1.6012 - val_rpn_class_l

Epoch 23/100
70/70 [==============================] - 912s 13s/step - loss: 0.8097 - rpn_class_loss: 0.0689 - rpn_bbox_loss: 0.1672 - mrcnn_class_loss: 0.2033 - mrcnn_bbox_loss: 0.1563 - mrcnn_mask_loss: 0.2141 - val_loss: 0.7285 - val_rpn_class_loss: 0.0609 - val_rpn_bbox_loss: 0.1419 - val_mrcnn_class_loss: 0.1874 - val_mrcnn_bbox_loss: 0.1361 - val_mrcnn_mask_loss: 0.2022
Epoch 24/100
70/70 [==============================] - 915s 13s/step - loss: 0.8067 - rpn_class_loss: 0.0627 - rpn_bbox_loss: 0.1687 - mrcnn_class_loss: 0.2110 - mrcnn_bbox_loss: 0.1561 - mrcnn_mask_loss: 0.2082 - val_loss: 0.7839 - val_rpn_class_loss: 0.0568 - val_rpn_bbox_loss: 0.1523 - val_mrcnn_class_loss: 0.2150 - val_mrcnn_bbox_loss: 0.1467 - val_mrcnn_mask_loss: 0.2131
Epoch 25/100
70/70 [==============================] - 915s 13s/step - loss: 0.7795 - rpn_class_loss: 0.0602 - rpn_bbox_loss: 0.1652 - mrcnn_class_loss: 0.2003 - mrcnn_bbox_loss: 0.1465 - mrcnn_mask_loss: 0.2073 - val_loss: 0.7244 - val_rpn_clas

KeyboardInterrupt: 